# Feature Engeneering

## Data loading/Загрузка данных

In [17]:
import os
from statistics import LinearRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_style("whitegrid")
np.random.seed(42)

In [19]:
DATA_PATH = "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [29]:
if os.path.exists(DATA_PATH):
    data = pd.read_csv(DATA_PATH)
    print(f"Данные загружены: {data.shape[0]} строк, {data.shape[1]} столбцов")
else:
    raise FileNotFoundError(f"Файл {DATA_PATH} не найден!")

Данные загружены: 7043 строк, 21 столбцов


In [30]:
# TotalCharges сильно коррелирует с tenure
# мы ее можем восстановить MonthlyCharges x tenure, поэтому избавимся от данного признака
data = data[data["tenure"] > 0] # т.к только пришедшие клиенты, не несут никакой информативности
data = data.drop("TotalCharges", axis=1)

In [31]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,Yes


In [32]:
drop_cols = ["customerID", "gender", "PhoneService", "MultipleLines"]
data = data.drop(drop_cols, axis=1)

In [33]:
numeric_cols = data.select_dtypes(include=np.number).columns.tolist()
categorical_cols = data.select_dtypes(include=["object", "str"]).columns.tolist()

numeric_cols.remove("SeniorCitizen")
categorical_cols.append("SeniorCitizen")

service_cols = [
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies"
]


for col in service_cols:
    data[col] = data[col].replace("No internet service", "No")
    data[col] = (data[col]== "Yes").astype(int)

In [47]:
data["Total_Services"] = data[service_cols].sum(axis=1)

In [49]:
data.head()

,SeniorCitizen,Partner,Dependents,tenure,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,Total_Services
0,0,Yes,No,1,DSL,0,1,0,0,0,0,Month-to-month,Yes,Electronic check,29.85,No,1
1,0,No,No,34,DSL,1,0,1,0,0,0,One year,No,Mailed check,56.95,No,2
2,0,No,No,2,DSL,1,1,0,0,0,0,Month-to-month,Yes,Mailed check,53.85,Yes,2
3,0,No,No,45,DSL,1,0,1,1,0,0,One year,No,Bank transfer (automatic),42.30,No,3
4,0,No,No,2,Fiber optic,0,0,0,0,0,0,Month-to-month,Yes,Electronic check,70.70,Yes,0


In [55]:
data["Contract"] = (data["Contract"] == "Month-to-month").astype(int)
data["PaymentMethod"] = (data["PaymentMethod"] == "Electronic check").astype(int)

In [56]:
data[pd.get_dummies(data["InternetService"]).drop("No", axis=1).columns] = pd.get_dummies(data["InternetService"]).drop("No", axis=1).astype(int)
data.drop("InternetService", axis=1, inplace=True)

In [57]:
binary_cols = ["Partner", "Dependents", "PaperlessBilling", "Churn"]
for col in binary_cols:
    data[col] = (data[col]== "Yes").astype(int)

In [58]:
data["is_new_client"] = (data["tenure"] < 6).astype("int")

In [59]:
data.head()

,SeniorCitizen,Partner,Dependents,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,Total_Services,is_new_client,DSL,Fiber optic
0,0,1,0,1,0,1,0,0,0,0,1,1,1,29.85,0,1,1,1,0
1,0,0,0,34,1,0,1,0,0,0,0,0,0,56.95,0,2,0,1,0
2,0,0,0,2,1,1,0,0,0,0,1,1,0,53.85,1,2,1,1,0
3,0,0,0,45,1,0,1,1,0,0,0,0,0,42.30,0,3,0,1,0
4,0,0,0,2,0,0,0,0,0,0,1,1,1,70.70,1,0,1,0,1


In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score

models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss")
}
results = {}

X = data.drop("Churn", axis=1)
y = data["Churn"]

# проверим баланс классов
print(f"Распределение классов:\n{y.value_counts(normalize=True)}")
print(f"Баланс классов:{y.value_counts(normalize=True)[1]:.1%} оттока")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # разделяем с сохранением распределения классов

print("="* 50)
print(f"Размер train выборки: {X_train.shape}")
print(f"Размер test выборки: {X_test.shape}")
print(f"Отток в train: {y_train.mean():.1%}")
print(f"Отток в test: {y_test.mean():.1%}")

Распределение классов:
Churn
0    0.734215
1    0.265785
Name: proportion, dtype: float64
Баланс классов:26.6% оттока
Размер train выборки: (5625, 18)
Размер test выборки: (1407, 18)
Отток в train: 26.6%
Отток в test: 26.6%


In [61]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        roc = roc_auc_score(y_test, y_pred_proba)
    else:
        roc = None

    print("\n📊 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

    print("\n📈 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    if roc:
        print(f"\n🎯 ROC-AUC Score: {roc:.4f}")
    return roc

In [63]:
for name, model in models.items():
    print(name)
    print("=" * 50)
    roc = evaluate_model(model, X_train, y_train, X_test, y_test)
    results[name] = {}

Logistic Regression

📊 Classification Report:
              precision    recall  f1-score   support

    No Churn       0.84      0.89      0.86      1033
       Churn       0.63      0.52      0.57       374

    accuracy                           0.79      1407
   macro avg       0.73      0.71      0.72      1407
weighted avg       0.78      0.79      0.79      1407


📈 Confusion Matrix:
[[919 114]
 [178 196]]

🎯 ROC-AUC Score: 0.8375
Random Forest

📊 Classification Report:
              precision    recall  f1-score   support

    No Churn       0.83      0.87      0.85      1033
       Churn       0.59      0.51      0.55       374

    accuracy                           0.78      1407
   macro avg       0.71      0.69      0.70      1407
weighted avg       0.77      0.78      0.77      1407


📈 Confusion Matrix:
[[903 130]
 [184 190]]

🎯 ROC-AUC Score: 0.8100
XGBoost

📊 Classification Report:
              precision    recall  f1-score   support

    No Churn       0.84      0.87